In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

This analysis takes an statistical inference approach to model the accidents data as a Poisson Process. The goal is to calculate an estimate of annual cost and loss of barrels of oil pipeline accidents in USA. So for the purposes of this analysis we reduce the dataset to a few required columns.

In [ ]:
data = pd.read_csv('/kaggle/input/pipeline-accidents/database.csv')
cols_of_interest = ['Accident Date/Time','Accident State','Pipeline Location','Liquid Type','Net Loss (Barrels)','All Costs']
data = data[cols_of_interest] #remove columns not needed for this analysis
print(data.head(5)) #see the top 5 recoords

Let us now calculate the summary statistics around our variables of interest in the sample i.e. costs incurred and loss of oil due to oil pipeline accidents.

In [ ]:
data_summary = data[['All Costs','Net Loss (Barrels)']].describe()
print(data_summary)

Looking at the summary statistics for both our variables presence of extreme values can easily be inferred because of extremely large MAX values as compared to their means and also significantly large standard deviations also indicates larger variance from the mean values of the variables. A Boxplot is a better way of visualizing this summary.

In [ ]:
import seaborn  as sns
import matplotlib.pyplot as plt

data['All Costs'] = data['All Costs'] / 1000000

plt.style.use('seaborn')

_ = sns.boxplot(data['All Costs'], data=data)
_ = plt.title('Cost of Accidents in (millions) US$')
plt.show()
plt.close()

__ = sns.boxplot(data['Net Loss (Barrels)'], data=data)
__ = plt.title('Net Loss (Barrels)')

plt.show()
plt.close()


The absence of the box in the box plot for both variables clearly indicate that the larger values for both variables are significantly larger than the rest of the sample and hence data for both variables have extreme values. Even though a median can be used to estimate costs per accident or loss in barrels per accident, the distribution of the data is highly unreliable. With such a distribution, fitting a model to the variable is difficult unless the extreme values can be eliminated. In our case, we do not have enough information to conclude that the extreme values are truly a data error and not part of the process. However, we have another key variable in the data sample which doesn’t immediately seem so important because a direct summary statistic cannot be carried out on it and that is the time for accident. With the knowledge of statistical modelling, we  can model the frequency of accidents in a certain interval with **Poisson Distribution** with the assumption that oil pipeline accidents in US do not occur simultaneously and that the occurrence of one accident has no impact on the occurrence of another accident. Then, the number accidents, X is a random variable with a Poisson Distribution X ~ Poisson(λ). 

In [ ]:
import numpy as np
data['Accident Date/Time'] = pd.to_datetime(data['Accident Date/Time'])
totaltimespan = np.max(data['Accident Date/Time']) - np.min(data['Accident Date/Time'])

totaltime_hour = (totaltimespan.days*24 + totaltimespan.seconds/(3600))
totaltime_month = (totaltimespan.days + totaltimespan.seconds/(3600*24)) *12/365

lmda_h = len(data) / totaltime_hour
lmda_m = len(data) / totaltime_month 

print('Estimated no. of accidents per hour: {}'.format(lmda_h))
print('Estimated no. of accidents per month: {}'.format(lmda_m))


Using NumPy in python, a random sample form a Poisson Distribution can then be generated to model our data with estimated λ to be 33 accidents per month. Let’s see how the distribution looks like and what can be inferred from it. 


In [ ]:
import math
import matplotlib.pyplot as plt

X = {} 
for i in range(66): #calculate probabilities for each value of n between 0 and 66
    X[i] = math.pow(2.71828,-1*33) * math.pow(33,i)/math.factorial(i)

p_poisson = pd.DataFrame(X.items(), columns=['X','PX'])

plt.style.use('seaborn')

fig = plt.subplots(figsize = (15,10))
_ = plt.plot(p_poisson['X'],p_poisson['PX'] , marker='.', color = 'purple', linestyle='solid')
plt.xlabel('Number of Accidents Per Month (n)')
plt.ylabel('P(X = n)')
plt.title('Probability Mass Function')
plt.show()

plt.close()

def cdf(data): #form a cumulative distribution function for data
    n = len(data)
    x = np.sort(data)
    y = np.arange(1, n+1) / n
    return x, y

np.random.seed(42)
samples_poisson = np.random.poisson(33,10000) #select a random sample of 3000 from a Poisson Distribution with mean = 33

x,y = cdf(samples_poisson) 

fig = plt.subplots(figsize = (15,10))
_ = plt.plot(x, y, marker='.', alpha = 0.5, color = 'purple', linestyle='solid')
plt.xlabel('Number of Accidents Per Month (n)')
plt.ylabel('P(X <= n)')
plt.title('Cumulative Distribution Function')
plt.show()

With the probability mass function, we now have a better conclusion about the data that even though the mean number of accidents per month is 33, **there is only a 7% chance of 33 accidents occurring per month**. Also, it is easy to see in the cumulative distribution function that **there is a 50% chance of 33 or less accidents occurring per month or there is a 50% chance of 33 or more accidents occurring per month which is not intuitive by only knowing the mean.** Statistical inference allows you to make more meaningful conclusions about your data. 

However, to estimate the number of oil pipeline accidents in US we are interested in mean time between accidents. The **Exponential Distribution** is sometimes used to model the time that elapses before an event occurs. In addition, if events follow a **Poisson Process** with a rate parameter of number of events per interval, then the waiting time from any starting point until the next event follows an exponential distribution with the same parameter of number of events per interval.  

Using Pandas, we can calculate time between accidents for each accident in our sample. Then we use NumPy to estimate mean time in hours between accidents as a parameter for exponential distribution. Then we plot the cumulative distribution function of our sample data with a random sample of size 3000 generated from an exponential distribution to compare how well our actual data fits the exponential distribution model.


In [ ]:
data.sort_values(by=['Accident Date/Time'], ascending = True, inplace = True)
data['timetoAccident'] = data['Accident Date/Time'].diff() #compute time between each accident
data['timetoAccident_h'] = data.apply(lambda x: x['timetoAccident'].days * 24 + x['timetoAccident'].seconds/3600, axis = 1)
data= data[data.timetoAccident.notnull()]

mean = np.mean(data['timetoAccident_h']) #parameter for exponential distribution

print('Mean time between accidents (h): {0}'.format(mean))



x,y = cdf(data['timetoAccident_h']) #cumulative distribution for accidents data

samples_exp =np.random.exponential(mean,size=3000) #random sample from exponential distribution
x_theor_exp,y_theor_exp = cdf(samples_exp)

fig = plt.subplots(figsize = (15,10))

_ = plt.plot(x, y, label = 'Actual Sample', marker='.', linestyle='none', color = 'purple')
_ = plt.plot(x_theor_exp, y_theor_exp,label = 'Exp Dist.', color = 'red', linestyle='solid')
_ = plt.legend(fontsize = 'large')
_ = plt.xlabel('Time between accidents in hours (x)')
_ = plt.ylabel('P(X <= x)')
plt.title('Cumulative Distribution Function')

plt.show()

plt.savefig('exp_cdf.png')
plt.close()

__ = plt.hist(samples_exp, bins = 50, density = True, histtype = 'step', color = 'red')
__ = plt.xlabel('Time between accidents in hours (x)')
__ = plt.ylabel('f(x)')
__ = plt.title('Probability Density Function')

plt.show()

The above graph shows the cumulative distributions for the sample data for accidents is very similar to the cumulative distribution of a random sample from an exponential distribution with the same mean as mean time to accidents. Hence, it would be appropriate to model time between oil pipeline accidents with an exponential distribution. This also solidifies our previous assumption that the occurrence of accidents is a Poisson Process with the number of accidents in disjoint intervals of time are independent and do not occur simultaneously. The probability density function of oil pipeline accidents sample is an exponential decay as expected from an exponentially distributed random variable.

So far, we have been able to calculate the mean time between oil pipeline accidents for our sample and have used it in its probabilistic model. But we still haven’t made any meaningful conclusions about the occurrence of any oil pipeline accidents in US. It is just not enough to know the mean of a sample and assume that it would apply to the entire population of accidents that happen in USA. So, how confident are we on the mean of our sample. If all the conditions of accident reporting and pipeline operations remained same during the past 50 years or past 100 years, would we get the same mean time to accidents if we had sampled data from 2002 to 2009 or any other time period. The intuitive assumption is that we would not get the exact same mean every time we sampled this data. Hence, lets apply the concept of **bootstrap sampling** to our sample and calculate a **confidence interval** around our sample mean.



In [ ]:
___ =plt.gca()
___.set_xlim([0,100])

for a in range(100):
    bs_sample = np.random.choice(data['timetoAccident_h'], size=len(data['timetoAccident_h']))
    x, y = cdf(bs_sample)
    ___ = plt.plot(x, y, marker='.', linestyle = 'none',
                 color='red', alpha=0.1)

x, y = cdf(data['timetoAccident_h'])
___ = plt.plot(x, y, label = 'Actual Sample', color = 'purple',linestyle = 'solid')

___ = plt.margins(0.02)
___ = plt.legend(fontsize = 'large')
___ = plt.xlabel('Time between accidents in hours (t)')
___ = plt.ylabel('P(T <= t)')
plt.title('Cumulative Distribution Function')
plt.show()

The distribution of 100 bootstrap samples is plotted on top of the actual data and as expected, the samples vary slightly from the actual sample while still closely maintaining the exponential distribution. The above plot has been zoomed to focus on the variation in samples. We now simply need the mean of each of our samples to compute a confidence interval on the sample mean. A statistic computed from a bootstrap sample is called a **bootstrap replicate**. 

In [ ]:
def bootstrap_replicate_1d(data, func):
    #Generate bootstrap replicate of 
    bs_sample = np.random.choice(data, len(data))
    return func(bs_sample)

def draw_bs_reps(data, func, size=1):
    #Draw bootstrap replicates

    bs_replicates = np.empty(size)
    
    for i in range(size):
        bs_replicates[i] = bootstrap_replicate_1d(data,func)

    return bs_replicates

bs_replicates = draw_bs_reps(data['timetoAccident_h'], np.mean, size=10000)

conf_int = np.percentile(bs_replicates,[2.5, 97.5])
print('95% confidence interval =', conf_int, 'hours')


____ = plt.hist(bs_replicates, bins=50, density=True, alpha = .8, color = 'red')
____ = plt.xlabel('Mean time to accident in hours (t)')
____ = plt.ylabel('P(T <= t)')
____ = plt.title('Probability Density Function')

plt.show()

The histogram in the above plot shows the probability density of the bootstrap replicates or the mean time to accidents when the process was repeated 10,000 times. For a 95% confidence interval, we need to find the range where 95% of times the mean of our replicates falls. That’s simply all values between the 2.5th and the 97.5th percentile of the bootstrap replicates as deduced from the probability density function of the bootstrap replicates.

With this analysis, we have 95% confidence that it takes between 21.15 and 22.93 hours for an oil pipeline accident to occur in the US or we have 95% confidence that about 382 to 414 oil pipeline accidents occur every year in the US. Using this knowledge, now we can infer the costs and the lost barrels due to oil pipeline accidents in US using the median value per incident for each variable. So, **we have 95% confidence that the annual cost incurred due to oil pipeline accidents in USA is between 8.9 to 9.6 million US dollars**. Now this is meaningful information that we learned about the oil pipeline accidents in US through statistical inference.
